# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Import Required Libraries 📚</h1></span>

In [1]:
%%writefile main.py
import os
import gc
import cv2
import math
import copy
import time
import random
import glob
from matplotlib import pyplot as plt

import h5py
from PIL import Image
from io import BytesIO

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
import torchvision

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

# For Image Models
import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

Writing main.py


In [2]:
%%writefile -a main.py
import sys
BEST_WEIGHT = sys.argv[1]
print(f"BEST_WEIGHT = {BEST_WEIGHT}")

Appending to main.py


# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Training Configuration ⚙️</h1></span>

In [3]:
%%writefile -a main.py
CONFIG = {
    "seed": 42,
    "img_size": 224,
    "model_name": "efficientnet_b0",
    "valid_batch_size": 32,
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
}

Appending to main.py


# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Set Seed for Reproducibility</h1></span>

In [4]:
%%writefile -a main.py
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG['seed'])

Appending to main.py


In [5]:
%%writefile -a main.py
ROOT_DIR = "/kaggle/input/isic-2024-challenge"
TEST_CSV = f'{ROOT_DIR}/test-metadata.csv'
TEST_HDF = f'{ROOT_DIR}/test-image.hdf5'
SAMPLE = f'{ROOT_DIR}/sample_submission.csv'

# BEST_WEIGHT = "/kaggle/input/efficientnet-weight/effnetv1b0_5_AUROC0.5924_Loss0.1916_epoch13.bin"

Appending to main.py


# <h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Read the Data 📖</h1>

In [6]:
%%writefile -a main.py
df = pd.read_csv(TEST_CSV)
df['target'] = 0 # dummy
df

Appending to main.py


In [7]:
%%writefile -a main.py
df_sub = pd.read_csv(SAMPLE)
df_sub

Appending to main.py


# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Dataset Class</h1></span>

In [8]:
%%writefile -a main.py
class ISICDataset(Dataset):
    def __init__(self, df, file_hdf, transforms=None):
        self.df = df
        self.fp_hdf = h5py.File(file_hdf, mode="r")
        self.isic_ids = df['isic_id'].values
        self.targets = df['target'].values
        self.transforms = transforms
        
    def __len__(self):
        return len(self.isic_ids)
    
    def __getitem__(self, index):
        isic_id = self.isic_ids[index]
        img = np.array( Image.open(BytesIO(self.fp_hdf[isic_id][()])) )
        target = self.targets[index]
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return {
            'image': img,
            'target': target,
        }

Appending to main.py


# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Augmentations</h1></span>

In [9]:
%%writefile -a main.py
data_transforms = {
    "valid": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.)
}

Appending to main.py


# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">GeM Pooling</h1></span>

In [10]:
%%writefile -a main.py
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6, p_clamp_min=1.0, p_clamp_max=6.0, dynamic_p=False):
        super(GeM, self).__init__()
        self.p = nn.Parameter(torch.ones(1) * p)  # 학습 가능한 p 파라미터 초기화
        self.eps = eps  # 작은 값으로 인해 0으로 나누는 것을 방지
        self.p_clamp_min = p_clamp_min
        self.p_clamp_max = p_clamp_max
        self.dynamic_p = dynamic_p
        
        if self.dynamic_p:
            # 동적으로 p 값을 계산하는 레이어 추가
            self.p_layer = nn.Sequential(
                nn.Conv2d(1, 1, kernel_size=1),
                nn.Sigmoid()
            )
    
    def forward(self, x):
        if self.dynamic_p:
            # p 값을 입력에 따라 동적으로 계산
            p = self.p_layer(x.mean(dim=(2, 3), keepdim=True))
            p = p * (self.p_clamp_max - self.p_clamp_min) + self.p_clamp_min  # p 값 범위 조정
        else:
            # 정적 p 값 사용
            p = self.p.clamp(min=self.p_clamp_min, max=self.p_clamp_max)
        
        return self.gem(x, p, self.eps)
        
    def gem(self, x, p, eps):
        # GeM 수식 적용
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
        
    def __repr__(self):
        # 클래스 설명 문자열 반환
        return (f"{self.__class__.__name__}(p={self.p.data.item():.4f}, eps={self.eps}, "
                f"p_clamp_min={self.p_clamp_min}, p_clamp_max={self.p_clamp_max}, "
                f"dynamic_p={self.dynamic_p})")



Appending to main.py


# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Create Model</h1></span>

In [11]:
%%writefile -a main.py
class ISICModel(nn.Module):
    def __init__(self, model_name, num_classes=1, pretrained=True):
        super(ISICModel, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)

        # EfficientNet-B1의 마지막 레이어 조정
        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Identity()
        self.model.global_pool = nn.Identity()
        
        # GeM Pooling 및 새로운 분류 레이어 추가
        self.pooling = GeM()
        self.linear = nn.Linear(in_features, num_classes)
        self.sigmoid = nn.Sigmoid() 

    def forward(self, images):
        features = self.model(images)
        pooled_features = self.pooling(features).flatten(1)
        output = self.linear(pooled_features)
        output = self.sigmoid(self.linear(pooled_features)) 
        return output
    
model = ISICModel('efficientnet_b0', pretrained=False)
model.load_state_dict( torch.load(BEST_WEIGHT) )
model.to(CONFIG['device']);

Appending to main.py


<span style="color: #000508; font-family: Segoe UI; font-size: 1.5em; font-weight: 300;">Prepare Dataloaders</span>

In [12]:
%%writefile -a main.py
test_dataset = ISICDataset(df, TEST_HDF, transforms=data_transforms["valid"])
test_loader = DataLoader(test_dataset, batch_size=CONFIG['valid_batch_size'], 
                          num_workers=2, shuffle=False, pin_memory=True)

Appending to main.py


<span style="color: #000508; font-family: Segoe UI; font-size: 1.5em; font-weight: 300;">Start Inference</span>

In [13]:
%%writefile -a main.py
preds = []
with torch.no_grad():
    bar = tqdm(enumerate(test_loader), total=len(test_loader))
    for step, data in bar:        
        images = data['image'].to(CONFIG["device"], dtype=torch.float)        
        batch_size = images.size(0)
        outputs = model(images)
        preds.append( outputs.detach().cpu().numpy() )
preds = np.concatenate(preds).flatten()

Appending to main.py


In [14]:
%%writefile -a main.py
df_sub["target"] = preds
df_sub.to_csv("submission.csv", index=False)

Appending to main.py


# Test

In [15]:
!python main.py /kaggle/input/efficientnet-weight/effnetv1b0_5_AUROC0.5924_Loss0.1916_epoch13.bin

/opt/conda/lib/python3.10/site-packages/albumentations/check_version.py:49: UserWarning: Error fetching version info <urlopen error [Errno -3] Temporary failure in name resolution>
  data = fetch_version_info()
BEST_WEIGHT = /kaggle/input/efficientnet-weight/effnetv1b0_5_AUROC0.5924_Loss0.1916_epoch13.bin
100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.02s/it]


In [16]:
!head submission.csv
!rm submission.csv

isic_id,target
ISIC_0015657,0.78303665
ISIC_0015729,0.037167512
ISIC_0015740,0.66878426
